## **Agile Project Simulation & Analysis**
## **Course: Software Project Management**
## **Author: Abdul Moiz Meer**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

sns.set_style("whitegrid")
plt.rcParams.update({'figure.figsize':(10,6), 'axes.titlesize':16, 'axes.labelsize':14, 'xtick.labelsize':12, 'ytick.labelsize':12})
os.makedirs("charts", exist_ok=True)
os.makedirs("tables", exist_ok=True)

### **1. Project Setup**

***Define Backlog,Teams,Risks***

In [2]:
backlog = pd.DataFrame({
    'StoryID': range(1, 201),
    'StoryPoints': np.random.randint(1, 13, 200),
    'TechDebtFactor': np.random.uniform(0, 0.2, 200),
    'Complexity': np.random.choice(['Small','Medium','Large'], 200),
    'Dependencies': [None]*200
})

teams = {
    'Alpha': {'AvgVelocity': 30, 'CostPerSprint': 5000},
    'Beta': {'AvgVelocity': 25, 'CostPerSprint': 4500},
    'Gamma': {'AvgVelocity': 20, 'CostPerSprint': 4000},
}

risks = pd.DataFrame({
    'Risk': ['Scope Creep','Key Member Departure','Integration Issue','Late Review','Tech Blocker'],
    'Probability': [0.2, 0.1, 0.15, 0.1, 0.05],
    'Impact': [0.2, 0.3, 0.25, 0.1, 0.15]
})

num_sprints = 20
critical_path = [1,2,3]

### **2. Simulation Functions**

In [3]:
def simulate_sprint(team_name, planned_velocity, tech_debt_factor, risk_events=[]):
    actual_velocity = np.random.normal(planned_velocity, planned_velocity*0.1)
    actual_velocity *= (1 - tech_debt_factor)
    for risk_impact in risk_events:
        actual_velocity *= (1 - risk_impact)
    return max(actual_velocity, 0)

def run_simulation(backlog, teams, risks, num_sprints, scope_change_sprint=5, extra_storypoints=50, critical_path=[]):
    remaining_backlog = backlog['StoryPoints'].sum()
    total_storypoints = remaining_backlog
    sprint_results = []
    team_velocity_records = {team: [] for team in teams.keys()}

    for sprint in range(1, num_sprints+1):
        if sprint == scope_change_sprint:
            remaining_backlog += extra_storypoints
            total_storypoints += extra_storypoints
        risk_events = [risk['Impact'] for idx, risk in risks.iterrows() if np.random.rand() < risk['Probability']]
        total_velocity = 0
        for team_name, info in teams.items():
            avg_tech_debt = backlog['TechDebtFactor'].mean()
            velocity = simulate_sprint(team_name, info['AvgVelocity'], avg_tech_debt, risk_events)
            team_velocity_records[team_name].append(velocity)
            total_velocity += velocity
        if critical_path:
            total_velocity *= 0.9
        remaining_backlog -= total_velocity
        sprint_results.append({'Sprint': sprint, 'CompletedPoints': total_velocity, 'RemainingPoints': max(remaining_backlog,0)})
        if remaining_backlog <= 0:
            break
    sprint_df = pd.DataFrame(sprint_results)
    sprint_df['PlannedValue'] = (sprint_df['Sprint']/num_sprints) * total_storypoints
    sprint_df['EarnedValue'] = total_storypoints - sprint_df['RemainingPoints']
    sprint_df['ScheduleVariance'] = sprint_df['EarnedValue'] - sprint_df['PlannedValue']
    return sprint_df, team_velocity_records, sprint

### **3. Monte Carlo Simulation**

In [4]:
num_runs = 100
completion_sprints = []

for run in range(num_runs):
    _, _, sprints_taken = run_simulation(backlog, teams, risks, num_sprints, critical_path=critical_path)
    completion_sprints.append(sprints_taken)

completion_array = np.array(completion_sprints)
completion_stats = {
    "Mean": np.mean(completion_array),
    "2.5_percentile": np.percentile(completion_array,2.5),
    "97.5_percentile": np.percentile(completion_array,97.5)
}
pd.DataFrame([completion_stats]).to_csv("tables/monte_carlo_completion.csv", index=False)

### **4. Visualization**

***4a: Histogram of Completion Sprints***

In [5]:
plt.figure()
counts, bins, patches = plt.hist(completion_array, bins=range(min(completion_array), max(completion_array)+2),
                                 color="teal", edgecolor='black')
plt.title("Distribution of Project Completion Sprints")
plt.xlabel("Number of Sprints")
plt.ylabel("Frequency")
for count, patch in zip(counts, patches):
    plt.text(patch.get_x()+patch.get_width()/2, count + 0.5, int(count), ha='center', va='bottom')
plt.grid(axis='y')
plt.savefig("charts/completion_histogram.png", dpi=300)
plt.close()

***4b: Detailed Sprint Simulation***

In [6]:
sprint_df, team_velocities, _ = run_simulation(backlog, teams, risks, num_sprints, critical_path=critical_path)
plt.figure()
plt.plot(sprint_df['Sprint'], sprint_df['RemainingPoints'], marker='o', color='crimson', linewidth=2)
for x, y in zip(sprint_df['Sprint'], sprint_df['RemainingPoints']):
    plt.text(x, y + 2, f"{int(y)}", ha='center', va='bottom', fontsize=10)
plt.axvline(x=5, color='orange', linestyle='--', label='Scope Change')
plt.title("Burndown Chart")
plt.xlabel("Sprint")
plt.ylabel("Remaining Story Points")
plt.legend()
plt.grid(True)
plt.savefig("charts/burndown_chart.png", dpi=300)
plt.close()

***4c: Team Velocity Trend***

In [7]:
plt.figure()
colors = ["navy", "forestgreen", "purple"]
for idx, (team, velocities) in enumerate(team_velocities.items()):
    plt.plot(range(1,len(velocities)+1), velocities, marker='o', label=team, color=colors[idx], linewidth=2)
    for x, y in zip(range(1,len(velocities)+1), velocities):
        plt.text(x, y + 0.5, f"{int(y)}", ha='center', va='bottom', fontsize=9)
plt.title("Team Velocity Trend per Sprint")
plt.xlabel("Sprint")
plt.ylabel("Velocity (SP)")
plt.legend()
plt.grid(True)
plt.savefig("charts/team_velocity_trend.png", dpi=300)
plt.close()

***4d: Earned Value Analysis***

In [8]:
plt.figure()
plt.plot(sprint_df['Sprint'], sprint_df['PlannedValue'], marker='o', label='Planned Value', color='darkorange', linewidth=2)
plt.plot(sprint_df['Sprint'], sprint_df['EarnedValue'], marker='x', label='Earned Value', color='darkblue', linewidth=2)
for x, y in zip(sprint_df['Sprint'], sprint_df['PlannedValue']):
    plt.text(x, y + 1, f"{int(y)}", ha='center', va='bottom', fontsize=9)
for x, y in zip(sprint_df['Sprint'], sprint_df['EarnedValue']):
    plt.text(x, y + 1, f"{int(y)}", ha='center', va='bottom', fontsize=9)
plt.title("Earned Value Analysis")
plt.xlabel("Sprint")
plt.ylabel("Story Points")
plt.legend()
plt.grid(True)
plt.savefig("charts/earned_value_analysis.png", dpi=300)
plt.close()

sprint_df.to_csv("tables/sprint_results.csv", index=False)

### **5. Sensitivity Analysis**

In [9]:
def sensitivity_analysis(backlog, teams, risks, num_sprints, variable, variation=0.2, num_runs=50):
    results = {}
    for change in [-variation, 0, variation]:
        modified_teams = {k:v.copy() for k,v in teams.items()}
        modified_backlog = backlog.copy()
        modified_risks = risks.copy()
        if variable == 'AlphaVelocity':
            modified_teams['Alpha']['AvgVelocity'] *= (1 + change)
        elif variable == 'BetaVelocity':
            modified_teams['Beta']['AvgVelocity'] *= (1 + change)
        elif variable == 'GammaVelocity':
            modified_teams['Gamma']['AvgVelocity'] *= (1 + change)
        elif variable == 'TechDebt':
            modified_backlog['TechDebtFactor'] *= (1 + change)
        elif variable == 'RiskProb':
            modified_risks.at[0,'Probability'] *= (1 + change)
        completion_sprints = []
        for run in range(num_runs):
            _, _, sprints_taken = run_simulation(modified_backlog, modified_teams, modified_risks, num_sprints, critical_path=critical_path)
            completion_sprints.append(sprints_taken)
        results[change] = np.mean(completion_sprints)
    return results

variables = ['AlphaVelocity','BetaVelocity','GammaVelocity','TechDebt','RiskProb']
sensitivity_results = {var: sensitivity_analysis(backlog, teams, risks, num_sprints, var) for var in variables}
sensitivity_df = pd.DataFrame({var: res for var, res in sensitivity_results.items()})
sensitivity_df.to_csv("tables/sensitivity_analysis.csv", index=False)

# Plot Sensitivity Analysis with data labels
plt.figure()
colors = ["red", "blue", "green", "purple", "orange"]
for idx, (var, result) in enumerate(sensitivity_results.items()):
    changes = [k*100 for k in result.keys()]
    mean_sprints = [v for v in result.values()]
    plt.plot(changes, mean_sprints, marker='o', label=var, color=colors[idx], linewidth=2)
    for x, y in zip(changes, mean_sprints):
        plt.text(x, y + 0.2, f"{y:.1f}", ha='center', va='bottom', fontsize=9)
plt.xlabel("Percent Change in Variable")
plt.ylabel("Mean Project Completion (Sprints)")
plt.title("Sensitivity Analysis of Key Variables")
plt.legend()
plt.grid(True)
plt.savefig("charts/sensitivity_analysis.png", dpi=300)
plt.close()